# Retaining Possession

In this notebook, we aim to analyze how successful teams are at retaining possession for at least 7 seconds based on where the throw in is located on the field.

In [178]:
# import libraries
from statsbombpy import sb
import json
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer.pitch import Pitch
import seaborn as sns
import os
import re

In [179]:
# obtain list of matches for specific competition and season 
df_matches = sb.matches(competition_id=55, season_id=43)
df_matches

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3795220,2021-07-06,21:00:00.000,Europe - UEFA Euro,2020,Italy,Spain,1,1,available,...,2022-08-04T12:00,6,Semi-finals,Wembley Stadium,Felix Brych,Roberto Mancini,Luis Enrique Martínez García,1.1.0,2,2
1,3788769,2021-06-21,21:00:00.000,Europe - UEFA Euro,2020,Russia,Denmark,1,4,available,...,2022-08-04T12:00,3,Group Stage,Parken,Clément Turpin,Stanislav Cherchesov,Kasper Hjulmand,1.1.0,2,2
2,3788761,2021-06-18,15:00:00.000,Europe - UEFA Euro,2020,Sweden,Slovakia,1,0,available,...,2022-08-04T12:00,2,Group Stage,Saint-Petersburg Stadium,Daniel Siebert,Jan Olof Andersson,Štefan Tarkovič,1.1.0,2,2
3,3788766,2021-06-20,18:00:00.000,Europe - UEFA Euro,2020,Italy,Wales,1,0,available,...,2022-08-04T12:00,3,Group Stage,Estadio Olímpico,Ovidiu Alin Hațegan,,,1.1.0,2,2
4,3788764,2021-06-19,18:00:00.000,Europe - UEFA Euro,2020,Portugal,Germany,2,4,available,...,2022-08-04T12:00,2,Group Stage,Allianz Arena,Anthony Taylor,Fernando Manuel Fernandes da Costa Santos,Joachim Löw,1.1.0,2,2
5,3788773,2021-06-23,21:00:00.000,Europe - UEFA Euro,2020,Portugal,France,2,2,available,...,2022-08-04T12:00,3,Group Stage,Puskás Aréna,Antonio Miguel Mateu Lahoz,Fernando Manuel Fernandes da Costa Santos,Didier Deschamps,1.1.0,2,2
6,3788755,2021-06-16,18:00:00.000,Europe - UEFA Euro,2020,Turkey,Wales,0,2,available,...,2022-08-04T12:00,2,Group Stage,Bakı Olimpiya Stadionu,Artur Manuel Ribeiro Soares Dias,,,1.1.0,2,2
7,3788754,2021-06-16,21:00:00.000,Europe - UEFA Euro,2020,Italy,Switzerland,3,0,available,...,2022-08-04T12:00,2,Group Stage,Estadio Olímpico,Sergey Karasev,Roberto Mancini,Vladimir Petković,1.1.0,2,2
8,3788762,2021-06-19,21:00:00.000,Europe - UEFA Euro,2020,Spain,Poland,1,1,available,...,2022-08-04T12:00,2,Group Stage,Estadio de La Cartuja,Daniele Orsato,Luis Enrique Martínez García,Paulo Manuel Carvalho de Sousa,1.1.0,2,2
9,3788760,2021-06-18,18:00:00.000,Europe - UEFA Euro,2020,Croatia,Czech Republic,1,1,available,...,2022-08-04T12:00,2,Group Stage,Hampden Park,Carlos del Cerro Grande,Zlatko Dalić,Jaroslav Šilhavý,1.1.0,2,2


In [180]:
# get all event info
df_events = pd.read_csv('full_data_sorted.csv')

/var/folders/fk/svllg0550bj0tsxddmfx5f1w0000gn/T/ipykernel_18981/355599449.py:2: DtypeWarning: Columns (2,4,12,26,35,43,49,69,85,86,87,88,89,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111) have mixed types. Specify dtype option on import or set low_memory=False.
  df_events = pd.read_csv('full_data_sorted.csv')


In [181]:
df_throwIn = df_events.loc[df_events['play_pattern'] == 'From Throw In']
df_throwIn

,Unnamed: 0,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,shot_saved_to_post,shot_redirect,pass_straight,goalkeeper_shot_saved_off_target,shot_saved_off_target,shot_follows_dribble,dribble_no_touch,goalkeeper_penalty_saved_to_post,goalkeeper_success_in_play,player_off_permanent
12,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,1067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,2088,NaN,NaN,NaN,NaN,"[3.3, 12.7]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,1068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192565,2803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192566,4023,NaN,NaN,NaN,NaN,"[70.4, 28.6]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192567,4633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192585,1419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
df_throwIn['timestamp']

12        00:00:12.163
13        00:00:14.093
14        00:00:14.093
15        00:00:16.420
16        00:00:18.242
              ...     
192565    00:13:58.761
192566    00:13:58.761
192567    00:13:59.242
192585    00:14:23.091
192586    00:14:25.224
Name: timestamp, Length: 42766, dtype: object

In [183]:
df_throwIn_new_index = df_throwIn.reset_index()
df_throwIn_final = df_throwIn_new_index.rename(columns={'level_0': 'full_index','Unnamed: 0': 'old_index'})
df_throwIn_final

,full_index,old_index,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,...,shot_saved_to_post,shot_redirect,pass_straight,goalkeeper_shot_saved_off_target,shot_saved_off_target,shot_follows_dribble,dribble_no_touch,goalkeeper_penalty_saved_to_post,goalkeeper_success_in_play,player_off_permanent
0,12,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13,1067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14,2088,NaN,NaN,NaN,NaN,"[3.3, 12.7]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,1068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42761,192565,2803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42762,192566,4023,NaN,NaN,NaN,NaN,"[70.4, 28.6]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42763,192567,4633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42764,192585,1419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
# create list of all competitions to analyze

directory = 'data/three-sixty'

# list of filenames
file_list = []
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        f_number = re.split("[/.]", f)[2]
        file_list.append(f_number)

file_list

['3788742',
 '3788754',
 '3794690',
 '3794686',
 '3788758',
 '3788774',
 '3788762',
 '3795107',
 '3788763',
 '3788775',
 '3788759',
 '3794687',
 '3794691',
 '3788755',
 '3788743',
 '3788748',
 '3788772',
 '3788764',
 '3795506',
 '3788744',
 '3788752',
 '3788768',
 '3788769',
 '3788753',
 '3788745',
 '3788765',
 '3788773',
 '3788749',
 '3788766',
 '3788770',
 '3788750',
 '3788746',
 '3788747',
 '3788751',
 '3788771',
 '3788767',
 '3795109',
 '3795220',
 '3788756',
 '3794688',
 '3788760',
 '3788776',
 '3795187',
 '3794692',
 '3794685',
 '3788761',
 '3788741',
 '3794689',
 '3788757',
 '3795221',
 '3795108']

In [185]:
# create dictionary deeming each throw in a success or failure
# success defined as retaining possession for at least 7 seconds 

from datetime import datetime

throw_in_outcomes = {}

current_possession = 0
i = 0

throwin_count = 0

for match in file_list:
    throwin_row = 0
    looking_for_result = False
    elapsed_time = 0
    current_possession = 0
    i = 0

    initial_match_df = df_throwIn_final[df_throwIn_final["match_id"]== int(match)]
    match_df = initial_match_df.reset_index()

    while i < len(match_df):
        if (match_df.loc[i, "pass_type"] == "Throw-in") and (looking_for_result == False):
            throwin_count += 1
            throwin_row = i
            elapsed_time = 0
            start_time = match_df.loc[i, "timestamp"]
            start_dt_time = datetime.strptime(start_time, '%H:%M:%S.%f')
            current_possession = match_df.loc[i, "possession"]
            looking_for_result = True
            # print(throwin_row)
        
        elif looking_for_result == True:
            end_time = match_df.loc[i, "timestamp"]
            end_dt_time = datetime.strptime(end_time, '%H:%M:%S.%f')
            elapsed_time = elapsed_time + (end_dt_time - start_dt_time).total_seconds()
            start_dt_time = end_dt_time

            if elapsed_time >= 7:
                if match_df.loc[i, "possession"] == current_possession:
                    throw_in_outcomes[throwin_count] = [match_df.loc[throwin_row, "location"], "Success"]
                else:
                    throw_in_outcomes[throwin_count] = [match_df.loc[throwin_row, "location"], "Failure"]
                    i = throwin_row
                looking_for_result = False
        
        i = i + 1

throw_in_outcomes




{1: ['[20.8, 0.1]', 'Success'],
 2: ['[59.2, 0.1]', 'Success'],
 3: ['[112.2, 0.1]', 'Success'],
 4: ['[26.8, 80.0]', 'Success'],
 5: ['[7.7, 80.0]', 'Failure'],
 6: ['[110.9, 0.1]', 'Failure'],
 7: ['[92.9, 80.0]', 'Failure'],
 8: ['[102.2, 80.0]', 'Failure'],
 9: ['[58.5, 0.1]', 'Failure'],
 10: ['[66.9, 80.0]', 'Failure'],
 11: ['[70.9, 0.1]', 'Failure'],
 12: ['[81.1, 0.1]', 'Success'],
 13: ['[48.4, 0.1]', 'Success'],
 14: ['[49.7, 0.1]', 'Success'],
 15: ['[49.1, 80.0]', 'Success'],
 16: ['[98.2, 0.1]', 'Success'],
 17: ['[99.9, 80.0]', 'Failure'],
 18: ['[83.8, 80.0]', 'Success'],
 19: ['[98.6, 0.1]', 'Success'],
 20: ['[87.8, 0.1]', 'Success'],
 21: ['[44.5, 80.0]', 'Success'],
 22: ['[112.1, 0.1]', 'Success'],
 23: ['[40.4, 0.1]', 'Failure'],
 24: ['[29.5, 0.1]', 'Success'],
 25: ['[91.6, 0.1]', 'Failure'],
 26: ['[110.7, 0.1]', 'Failure'],
 27: ['[108.7, 0.1]', 'Success'],
 28: ['[26.1, 0.1]', 'Failure'],
 29: ['[72.5, 0.1]', 'Failure'],
 30: ['[56.7, 80.0]', 'Success'],
 31:

# Analysis attempt: linear regression

In [129]:
# linear regression
import numpy as np
from sklearn.linear_model import LinearRegression

list_of_x_coordinates = []
for throwin in throw_in_outcomes:
    coords = throw_in_outcomes[throwin][0]
    x_coord = coords.split(",")[0][1:]
    list_of_x_coordinates.append(float(x_coord))

list_of_outcomes = []
for throwin in throw_in_outcomes:
    outcome = throw_in_outcomes[throwin][1]
    if outcome == "Success":
        outcome_val = 1
    else:
        outcome_val = 0

    list_of_outcomes.append(outcome_val)

x = np.array(list_of_x_coordinates).reshape((-1, 1))
print(x)

y = np.array(list_of_outcomes)
print(y)


[[ 20.8]
 [ 59.2]
 [112.2]
 ...
 [106.4]
 [ 36.2]
 [ 54.8]]
[1 1 1 ... 1 1 0]


In [130]:
# modeling

model = LinearRegression()

model.fit(x, y)

LinearRegression()

In [131]:
r_sq = model.score(x, y)
print(f"coefficient of determination: {r_sq}")
print(f"intercept: {model.intercept_}")
print(f"slope: {model.coef_}")

coefficient of determination: 0.014712869011338503
intercept: 0.5429441047435098
slope: [0.00195444]


In [132]:
y_pred = model.predict(x)
print(f"predicted response:\n{y_pred}")

predicted response:
[0.58359654 0.6586472  0.76223275 ... 0.75089697 0.61369499 0.65004765]


That didn't work... try with only data from half the field (don't think linear shape is working, maybe just using half the field would yield more of a linear trend?)

In [133]:
list_of_x_coordinates = []
list_of_outcomes = []

for throwin in throw_in_outcomes:
    coords = throw_in_outcomes[throwin][0]
    x_coord = coords.split(",")[0][1:]
    if float(x_coord) <= 60:
        list_of_x_coordinates.append(float(x_coord))
        outcome = throw_in_outcomes[throwin][1]
        if outcome == "Success":
            outcome_val = 1
        else:
            outcome_val = 0
        list_of_outcomes.append(outcome_val)

x = np.array(list_of_x_coordinates).reshape((-1, 1))
print(x)

y = np.array(list_of_outcomes)
print(y)

[[20.8]
 [59.2]
 [26.8]
 [ 7.7]
 [58.5]
 [48.4]
 [49.7]
 [49.1]
 [44.5]
 [40.4]
 [29.5]
 [26.1]
 [56.7]
 [45.4]
 [43.5]
 [55. ]
 [38.6]
 [ 2.7]
 [57.2]
 [23.9]
 [26.8]
 [43.3]
 [55.4]
 [55.8]
 [38.9]
 [57.7]
 [25.7]
 [21.1]
 [ 8.5]
 [20.8]
 [58.4]
 [ 8.3]
 [31.8]
 [52.4]
 [48.4]
 [42.6]
 [54.5]
 [49.5]
 [22.1]
 [48. ]
 [19.8]
 [43.3]
 [49.6]
 [34.6]
 [48.6]
 [56.3]
 [20.6]
 [55.9]
 [21.9]
 [34.3]
 [26.6]
 [14. ]
 [33.4]
 [41.3]
 [34.3]
 [25.5]
 [57.1]
 [21.6]
 [55.8]
 [34.1]
 [42.3]
 [38.3]
 [34.7]
 [26.1]
 [43.1]
 [33.2]
 [53. ]
 [46.4]
 [11.9]
 [39.2]
 [38.3]
 [57.1]
 [22.4]
 [50.1]
 [55.2]
 [43.4]
 [41.6]
 [47.8]
 [39.8]
 [48.1]
 [ 9.3]
 [10.2]
 [49.2]
 [44.3]
 [53.6]
 [56.6]
 [11. ]
 [57.5]
 [31. ]
 [22.9]
 [29.1]
 [30.4]
 [41.8]
 [27.7]
 [37.9]
 [23.4]
 [25.9]
 [55.4]
 [43.6]
 [36.7]
 [ 3.2]
 [22.8]
 [14.6]
 [51.8]
 [22.9]
 [26.6]
 [22.1]
 [35.3]
 [52.3]
 [41.3]
 [44.1]
 [34.5]
 [36. ]
 [24.4]
 [36.1]
 [50.1]
 [19.8]
 [24.4]
 [35.1]
 [13.3]
 [42. ]
 [19.3]
 [49.7]
 [29.3]
 [10.9]


In [134]:
# new model

new_model = LinearRegression()

new_model.fit(x, y)

LinearRegression()

In [135]:
r_sq = model.score(x, y)
print(f"coefficient of determination: {r_sq}")
print(f"intercept: {model.intercept_}")
print(f"slope: {model.coef_}")

coefficient of determination: 0.008211759760914483
intercept: 0.5429441047435098
slope: [0.00195444]


In [136]:
y_pred = model.predict(x)
print(f"predicted response:\n{y_pred}")

predicted response:
[0.58359654 0.6586472  0.59532321 0.55799333 0.65727909 0.63753921
 0.64007998 0.63890732 0.62991687 0.62190365 0.60060021 0.5939551
 0.65376109 0.63167587 0.62796243 0.65043854 0.61838565 0.5482211
 0.65473831 0.58965532 0.59532321 0.62757154 0.65122032 0.65200209
 0.61897199 0.65571554 0.59317332 0.58418288 0.55955688 0.58359654
 0.65708365 0.55916599 0.60509543 0.64535698 0.63753921 0.62620343
 0.64946132 0.63968909 0.58613732 0.63675743 0.5816421  0.62757154
 0.63988454 0.61056788 0.63793009 0.65297931 0.58320566 0.65219754
 0.58574643 0.60998154 0.59493232 0.57030632 0.60822254 0.62366265
 0.60998154 0.59278243 0.65454287 0.5851601  0.65200209 0.60959065
 0.6256171  0.61779932 0.61076332 0.5939551  0.62718065 0.60783165
 0.64652965 0.63363032 0.56620199 0.61955832 0.61779932 0.65454287
 0.58672366 0.64086176 0.65082943 0.62776698 0.62424898 0.63636654
 0.62073099 0.63695287 0.56112044 0.56287944 0.63910276 0.62952598
 0.64770232 0.65356565 0.56444299 0.65532465

In [137]:
# with statsmodels
x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     9.090
Date:                Tue, 16 Aug 2022   Prob (F-statistic):            0.00265
Time:                        22:05:26   Log-Likelihood:                -573.67
No. Observations:                 822   AIC:                             1151.
Df Residuals:                     820   BIC:                             1161.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4717      0.047      9.983      0.0

That's worse... trying multiple linear regression

In [161]:
# create dictionary deeming each throw in a success or failure and including other info
# success defined as retaining possession for at least 7 seconds 

from datetime import datetime

throw_in_outcomes_extended = {}

current_possession = 0
i = 0

throwin_count = 0

for match in file_list:
    throwin_row = 0
    looking_for_result = False
    elapsed_time = 0
    current_possession = 0
    i = 0

    initial_match_df = df_throwIn_final[df_throwIn_final["match_id"]== int(match)]
    match_df = initial_match_df.reset_index()

    while i < len(match_df):
        if (match_df.loc[i, "pass_type"] == "Throw-in") and (looking_for_result == False):
            throwin_count += 1
            throwin_row = i
            elapsed_time = 0
            start_time = match_df.loc[i, "timestamp"]
            start_dt_time = datetime.strptime(start_time, '%H:%M:%S.%f')
            current_possession = match_df.loc[i, "possession"]
            looking_for_result = True
            # print(throwin_row)
        
        elif looking_for_result == True:
            end_time = match_df.loc[i, "timestamp"]
            end_dt_time = datetime.strptime(end_time, '%H:%M:%S.%f')
            elapsed_time = elapsed_time + (end_dt_time - start_dt_time).total_seconds()
            start_dt_time = end_dt_time

            if elapsed_time >= 7:
                
                if match_df.loc[i, "possession"] == current_possession:
                    throw_in_outcomes_extended[throwin_count] = [match_df.loc[throwin_row, "location"], match_df.loc[throwin_row, "pass_height"], match_df.loc[throwin_row, "pass_angle"], match_df.loc[throwin_row, "pass_length"], "Success"]
                
                else:
                    throw_in_outcomes_extended[throwin_count] = [match_df.loc[throwin_row, "location"], match_df.loc[throwin_row, "pass_height"], match_df.loc[throwin_row, "pass_angle"], match_df.loc[throwin_row, "pass_length"], "Failure"]
                    
                    i = throwin_row
                
                looking_for_result = False
        
        i = i + 1

throw_in_outcomes_extended




{1: ['[20.8, 0.1]', 'High Pass', 0.24497867, 24.738634, 'Success'],
 2: ['[59.2, 0.1]', 'High Pass', 0.8421932, 21.176638, 'Success'],
 3: ['[112.2, 0.1]', 'High Pass', 1.6303848, 35.262585, 'Success'],
 4: ['[26.8, 80.0]', 'Low Pass', -0.63723814, 9.580188, 'Success'],
 5: ['[7.7, 80.0]', 'Low Pass', -0.16154292, 13.678084, 'Failure'],
 6: ['[110.9, 0.1]', 'Low Pass', 0.84593016, 9.350936, 'Failure'],
 7: ['[92.9, 80.0]', 'Low Pass', -0.29059917, 11.168259, 'Failure'],
 8: ['[102.2, 80.0]', 'High Pass', -1.3639224, 41.382362, 'Failure'],
 9: ['[58.5, 0.1]', 'High Pass', 0.94115114, 17.321663, 'Failure'],
 10: ['[66.9, 80.0]', 'Low Pass', -0.12161902, 9.066973, 'Failure'],
 11: ['[70.9, 0.1]', 'Low Pass', 0.5052902, 21.484879, 'Failure'],
 12: ['[81.1, 0.1]', 'Low Pass', 2.8069603, 39.278748, 'Success'],
 13: ['[48.4, 0.1]', 'Low Pass', 1.1156955, 10.46518, 'Success'],
 14: ['[49.7, 0.1]', 'High Pass', 0.19228487, 23.024336, 'Success'],
 15: ['[49.1, 80.0]', 'Low Pass', -0.46869808, 8.

In [162]:
full_list = []

list_of_x_coordinates = []

for throwin in throw_in_outcomes_extended:
    coords = throw_in_outcomes_extended[throwin][0]
    x_coord = coords.split(",")[0][1:]
    list_of_x_coordinates.append(float(x_coord))
    if throw_in_outcomes_extended[throwin][1] == "Low Pass":
        list_of_x_coordinates.append(0)
    else:
        list_of_x_coordinates.append(1)
    list_of_x_coordinates += (throw_in_outcomes_extended[throwin][2:4])
    full_list.append(list_of_x_coordinates)
    list_of_x_coordinates = []


list_of_outcomes = []
for throwin in throw_in_outcomes_extended:
    outcome = throw_in_outcomes_extended[throwin][4]
    if outcome == "Success":
        outcome_val = 1
    else:
        outcome_val = 0
    list_of_outcomes.append(outcome_val)

print(full_list)

[[20.8, 1, 0.24497867, 24.738634], [59.2, 1, 0.8421932, 21.176638], [112.2, 1, 1.6303848, 35.262585], [26.8, 0, -0.63723814, 9.580188], [7.7, 0, -0.16154292, 13.678084], [110.9, 0, 0.84593016, 9.350936], [92.9, 0, -0.29059917, 11.168259], [102.2, 1, -1.3639224, 41.382362], [58.5, 1, 0.94115114, 17.321663], [66.9, 0, -0.12161902, 9.066973], [70.9, 0, 0.5052902, 21.484879], [81.1, 0, 2.8069603, 39.278748], [48.4, 0, 1.1156955, 10.46518], [49.7, 1, 0.19228487, 23.024336], [49.1, 0, -0.46869808, 8.854942], [98.2, 0, 0.24497867, 10.720075], [99.9, 0, -1.6451272, 9.426028], [83.8, 0, -2.5503006, 8.431489], [98.6, 0, 0.5255838, 5.7801385], [87.8, 0, 2.8440883, 22.174084], [44.5, 0, -2.4420047, 13.200758], [112.1, 0, 1.9088625, 13.567977], [40.4, 1, 0.28435054, 43.132008], [29.5, 0, 1.5707964, 12.4], [91.6, 0, 0.90602183, 14.103191], [110.7, 0, 2.3855977, 9.6208105], [108.7, 0, 2.4932606, 12.420145], [26.1, 1, 0.25405753, 18.700268], [72.5, 1, 0.11710875, 30.811037], [56.7, 1, -0.33339298, 24.

In [163]:
details_df = pd.DataFrame(full_list, columns = ['location', 'height', 'angle', 'distance'])

details_df

,location,height,angle,distance
0,20.8,1,0.244979,24.738634
1,59.2,1,0.842193,21.176638
2,112.2,1,1.630385,35.262585
3,26.8,0,-0.637238,9.580188
4,7.7,0,-0.161543,13.678084
...,...,...,...,...
1893,26.2,0,2.325277,6.862215
1894,106.8,1,-1.432344,12.317873
1895,106.4,1,-1.482073,32.728733
1896,36.2,0,2.527574,14.926487


In [164]:
x = np.asarray(details_df)
x

array([[ 20.8       ,   1.        ,   0.24497867,  24.738634  ],
       [ 59.2       ,   1.        ,   0.8421932 ,  21.176638  ],
       [112.2       ,   1.        ,   1.6303848 ,  35.262585  ],
       ...,
       [106.4       ,   1.        ,  -1.4820728 ,  32.728733  ],
       [ 36.2       ,   0.        ,   2.5275736 ,  14.926487  ],
       [ 54.8       ,   1.        ,   1.0309176 ,  31.710093  ]])

In [165]:
y = np.array(list_of_outcomes)
print(y)

[1 1 1 ... 1 1 0]


In [166]:
import statsmodels.api as sm
model = sm.OLS(y, x)

In [167]:
print(x.shape, y.shape)

(1898, 4) (1898,)


In [168]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [169]:
model = LinearRegression()
model.fit(X_train, Y_train)

LinearRegression()

In [170]:
r_sq = model.score(x,y)
print(r_sq)

0.06547036494981684


In [171]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(x, y)

LinearRegression()

In [172]:
print(regr.coef_)

[ 0.00146577 -0.26240988 -0.00663975  0.00679311]


In [173]:
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 0.5160421558494432
Coefficients: 
 [ 0.00146577 -0.26240988 -0.00663975  0.00679311]


In [174]:
# with statsmodels
x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     33.34
Date:                Tue, 16 Aug 2022   Prob (F-statistic):           6.54e-27
Time:                        22:08:39   Log-Likelihood:                -1194.6
No. Observations:                1898   AIC:                             2399.
Df Residuals:                    1893   BIC:                             2427.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5160      0.035     14.719      0.0

# General data analysis

In [186]:
# how many successful throw-ins?

throwin_total = 0
successful_total = 0

for throwin in throw_in_outcomes:
    throwin_total += 1
    outcome = throw_in_outcomes[throwin][1]
    if outcome == "Success":
        successful_total += 1

success_percentage = successful_total/throwin_total

print("Overall success rate: " + str(success_percentage))


Overall success rate: 0.6712328767123288


In [187]:
# Throw-ins from which part of the field lead to the most success?

# split location into 4: penalty area (18 yards) on both sides and down the middle 
# area 1 (0-18 and 102-120): penalty area, area 2 (18-102) attacking area

penalty_total = 0
middle_total = 0

penalty_successes = 0
middle_successes = 0

for throwin in throw_in_outcomes:
    coords = throw_in_outcomes[throwin][0]
    x_coord = float(coords.split(",")[0][1:])
    outcome = throw_in_outcomes[throwin][1]

    if x_coord >= 102 or x_coord <= 18:
        middle_total += 1
        if outcome == "Success":
            middle_successes += 1
    
    else:
        penalty_total += 1
        if outcome == "Success":
            penalty_successes += 1

penalty_success_rate = penalty_successes/penalty_total

middle_success_rate = middle_successes/middle_total

print("Penalty area success rate: " + str(penalty_successes) + "/" + str(penalty_total) + " = " + str(penalty_success_rate))
print("Attacking area success rate: "  + str(middle_successes) + "/" + str(middle_total) + " = " +  str(middle_success_rate))

Penalty area success rate: 1047/1549 = 0.6759199483537767
Attacking area success rate: 227/349 = 0.6504297994269341


In [83]:
from prettytable import PrettyTable

myTable = PrettyTable(["Throw-in Location", "Successes", "Total", "Success Rate"])

myTable.add_row(["Penalty", penalty_successes, penalty_total, round(penalty_success_rate, 4)])
myTable.add_row(["Attacking", middle_successes, middle_total, round(middle_success_rate, 4)])

print(myTable)

+-------------------+-----------+-------+--------------+
| Throw-in Location | Successes | Total | Success Rate |
+-------------------+-----------+-------+--------------+
|      Penalty      |    1047   |  1549 |    0.6759    |
|     Attacking     |    227    |  349  |    0.6504    |
+-------------------+-----------+-------+--------------+


For our data on retaining possession, we found that teams throwing in the ball were roughly equally likely to retain possession in both the penalty areas (67.59%) and middle areas (65.04%) of the field, although perhaps slightly more likely to retain possession in the penalty areas. 

**Which teams are best at retaining possession?**

In [188]:
# create dictionary deeming each throw in a success or failure and including which team threw in the ball
# success defined as retaining possession for at least 7 seconds 

from datetime import datetime

throw_in_outcomes_team = {}

current_possession = 0
i = 0

throwin_count = 0

for match in file_list:
    throwin_row = 0
    looking_for_result = False
    elapsed_time = 0
    current_possession = 0
    i = 0

    initial_match_df = df_throwIn_final[df_throwIn_final["match_id"]== int(match)]
    match_df = initial_match_df.reset_index()

    while i < len(match_df):
        if (match_df.loc[i, "pass_type"] == "Throw-in") and (looking_for_result == False):
            throwin_count += 1
            throwin_row = i
            elapsed_time = 0
            start_time = match_df.loc[i, "timestamp"]
            start_dt_time = datetime.strptime(start_time, '%H:%M:%S.%f')
            current_possession = match_df.loc[i, "possession"]
            looking_for_result = True
            # print(throwin_row)
        
        elif looking_for_result == True:
            end_time = match_df.loc[i, "timestamp"]
            end_dt_time = datetime.strptime(end_time, '%H:%M:%S.%f')
            elapsed_time = elapsed_time + (end_dt_time - start_dt_time).total_seconds()
            start_dt_time = end_dt_time

            if elapsed_time >= 7:
                
                if match_df.loc[i, "possession"] == current_possession:
                    throw_in_outcomes_team[throwin_count] = [match_df.loc[throwin_row, "possession_team"], "Success"]
                
                else:
                    throw_in_outcomes_team[throwin_count] = [match_df.loc[throwin_row, "possession_team"], "Failure"]
                    
                    i = throwin_row
                
                looking_for_result = False
        
        i = i + 1

throw_in_outcomes_team

{1: ['Finland', 'Success'],
 2: ['Finland', 'Success'],
 3: ['Finland', 'Success'],
 4: ['Denmark', 'Success'],
 5: ['Finland', 'Failure'],
 6: ['Denmark', 'Failure'],
 7: ['Finland', 'Failure'],
 8: ['Finland', 'Failure'],
 9: ['Finland', 'Failure'],
 10: ['Finland', 'Failure'],
 11: ['Finland', 'Failure'],
 12: ['Finland', 'Success'],
 13: ['Finland', 'Success'],
 14: ['Denmark', 'Success'],
 15: ['Finland', 'Success'],
 16: ['Denmark', 'Success'],
 17: ['Denmark', 'Failure'],
 18: ['Denmark', 'Success'],
 19: ['Denmark', 'Success'],
 20: ['Denmark', 'Success'],
 21: ['Denmark', 'Success'],
 22: ['Denmark', 'Success'],
 23: ['Finland', 'Failure'],
 24: ['Denmark', 'Success'],
 25: ['Denmark', 'Failure'],
 26: ['Denmark', 'Failure'],
 27: ['Denmark', 'Success'],
 28: ['Finland', 'Failure'],
 29: ['Finland', 'Failure'],
 30: ['Finland', 'Success'],
 31: ['Finland', 'Failure'],
 32: ['Denmark', 'Success'],
 33: ['Denmark', 'Success'],
 34: ['Denmark', 'Success'],
 35: ['Finland', 'Failu

In [189]:
match_df.columns

Index(['level_0', 'full_index', 'old_index', 'ball_receipt_outcome',
       'ball_recovery_offensive', 'ball_recovery_recovery_failure',
       'block_deflection', 'carry_end_location', 'clearance_aerial_won',
       'clearance_body_part',
       ...
       'shot_saved_to_post', 'shot_redirect', 'pass_straight',
       'goalkeeper_shot_saved_off_target', 'shot_saved_off_target',
       'shot_follows_dribble', 'dribble_no_touch',
       'goalkeeper_penalty_saved_to_post', 'goalkeeper_success_in_play',
       'player_off_permanent'],
      dtype='object', length=114)

In [190]:
team_dict = {}

for attempt in throw_in_outcomes_team.values():
    team_name = attempt[0]
    
    if attempt[0] not in team_dict:
        if attempt[1] == "Success":
            team_dict[team_name] = [1, 0]
        else:
            team_dict[team_name] = [0, 1]
    
    else:
        if attempt[1] == "Success":
            team_dict[team_name][0] += 1
        else:
            team_dict[team_name][1] += 1

team_dict

{'Finland': [21, 19],
 'Denmark': [69, 37],
 'Switzerland': [60, 27],
 'Italy': [115, 47],
 'Netherlands': [52, 22],
 'Czech Republic': [59, 44],
 'Croatia': [52, 29],
 'Spain': [94, 27],
 'North Macedonia': [33, 35],
 'Ukraine': [58, 34],
 'Germany': [59, 8],
 'Hungary': [29, 21],
 'Poland': [48, 20],
 'Belgium': [66, 26],
 'France': [46, 24],
 'Slovakia': [30, 18],
 'England': [96, 42],
 'Scotland': [31, 22],
 'Portugal': [44, 19],
 'Turkey': [29, 5],
 'Wales': [41, 20],
 'Russia': [35, 22],
 'Sweden': [45, 30],
 'Austria': [62, 26]}

In [191]:
team_dict_updated = {}

for team in team_dict.keys():
    team_dict_updated[team] = (team_dict[team][0] / (team_dict[team][0] + team_dict[team][1]))

team_dict_updated

{'Finland': 0.525,
 'Denmark': 0.6509433962264151,
 'Switzerland': 0.6896551724137931,
 'Italy': 0.7098765432098766,
 'Netherlands': 0.7027027027027027,
 'Czech Republic': 0.5728155339805825,
 'Croatia': 0.6419753086419753,
 'Spain': 0.7768595041322314,
 'North Macedonia': 0.4852941176470588,
 'Ukraine': 0.6304347826086957,
 'Germany': 0.8805970149253731,
 'Hungary': 0.58,
 'Poland': 0.7058823529411765,
 'Belgium': 0.717391304347826,
 'France': 0.6571428571428571,
 'Slovakia': 0.625,
 'England': 0.6956521739130435,
 'Scotland': 0.5849056603773585,
 'Portugal': 0.6984126984126984,
 'Turkey': 0.8529411764705882,
 'Wales': 0.6721311475409836,
 'Russia': 0.6140350877192983,
 'Sweden': 0.6,
 'Austria': 0.7045454545454546}

In [192]:
from prettytable import PrettyTable

team_table = PrettyTable(["Team", "Possession retention success rate"])

for team in team_dict_updated.keys():
    team_table.add_row([team, round(team_dict_updated[team], 4)])

print(team_table.get_string(sortby="Possession retention success rate", reversesort=True))


+-----------------+-----------------------------------+
|       Team      | Possession retention success rate |
+-----------------+-----------------------------------+
|     Germany     |               0.8806              |
|      Turkey     |               0.8529              |
|      Spain      |               0.7769              |
|     Belgium     |               0.7174              |
|      Italy      |               0.7099              |
|      Poland     |               0.7059              |
|     Austria     |               0.7045              |
|   Netherlands   |               0.7027              |
|     Portugal    |               0.6984              |
|     England     |               0.6957              |
|   Switzerland   |               0.6897              |
|      Wales      |               0.6721              |
|      France     |               0.6571              |
|     Denmark     |               0.6509              |
|     Croatia     |               0.642         